# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import sys
sys.path.append("..")

In [2]:
import os
import glob

import psycopg2
import pandas as pd

from data_modelling import sql_queries
from data_modelling.db import create_connection
from data_modelling.etl_steps.read import get_files, read_file

In [3]:
conn = create_connection("sparkifydb")
cur = conn.cursor()

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("../data/song_data/")

In [5]:
filepath = song_files[0]

In [6]:
df = read_file(filepath)
df

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNNKDK1187B98BBD5,45.80726,Zagreb Croatia,15.9676,Jinx,407.37914,1,SOFNOQK12AB01840FC,Kutt Free (DJ Volume Remix),0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
from data_modelling.etl_steps.prepare import PreparerSongs, PreparerArtists, PreparerTime, PreparerUsers, PreparerSongplaysStaging

In [8]:
preparer_songs = PreparerSongs()

In [9]:
song_data = preparer_songs.transform(df)
song_data

[{'song_id': 'SOFNOQK12AB01840FC',
  'title': 'Kutt Free (DJ Volume Remix)',
  'artist_id': 'ARNNKDK1187B98BBD5',
  'year': 0,
  'duration': 407.37914}]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [10]:
from psycopg2.errors import UniqueViolation, NotNullViolation

In [11]:
def insert(query, records, cur, conn):
    for record in records:
        success = True
        try:
            cur.execute(query, record)
        except UniqueViolation:
            print("Record violates UNIQUE constraint.")
            success = False
        except NotNullViolation:
            print("Record violates NOT NULL constraint.")
            success = False
        except Exception as error:
            print("An unkown error has occured.")
            print(error)
            success = False
        if success:
            print("Record inserted successfully.")
        conn.commit()

In [12]:
insert(sql_queries.song_table_insert, song_data, cur, conn)

Record inserted successfully.


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [13]:
preparer_artists = PreparerArtists()

In [14]:
artist_data = preparer_artists.transform(df)
artist_data

[{'artist_id': 'ARNNKDK1187B98BBD5',
  'name': 'Jinx',
  'location': 'Zagreb Croatia',
  'lattitude': 45.80726,
  'longitude': 15.967600000000001}]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [15]:
insert(sql_queries.artist_table_insert, artist_data, cur, conn)

Record inserted successfully.


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [16]:
log_files = get_files("../data/log_data/")

In [17]:
filepath = log_files[0]

In [18]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Dominick,M,0,Norris,NaN,free,"Los Angeles-Long Beach-Anaheim, CA",GET,Home,1.540976e+12,44,None,200,1541635950796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",45
1,Slipknot,Logged In,Aiden,M,0,Ramirez,192.57424,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Opium Of The People (Album Version),200,1541639510796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
2,Cat Stevens,Logged In,Aiden,M,1,Ramirez,170.57914,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Don't Be Shy,200,1541639702796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
3,Collective Soul,Logged In,Aiden,M,2,Ramirez,273.47546,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Run (LP Version),200,1541639872796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
4,Taylor Swift,Logged In,Aiden,M,3,Ramirez,233.89995,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Love Story,200,1541640145796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [19]:
df = df[df["page"] == "NextSong"]
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Slipknot,Logged In,Aiden,M,0,Ramirez,192.57424,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Opium Of The People (Album Version),200,1541639510796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
2,Cat Stevens,Logged In,Aiden,M,1,Ramirez,170.57914,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Don't Be Shy,200,1541639702796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
3,Collective Soul,Logged In,Aiden,M,2,Ramirez,273.47546,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Run (LP Version),200,1541639872796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
4,Taylor Swift,Logged In,Aiden,M,3,Ramirez,233.89995,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Love Story,200,1541640145796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
5,The Dixie Cups,Logged In,Aiden,M,4,Ramirez,120.05832,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Iko Iko,200,1541640378796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20


In [20]:
preparer_time = PreparerTime()

In [21]:
time_data = preparer_time.transform(df)

In [22]:
time_data

[{'start_time': 1541639510796,
  'hour': 1,
  'day': 8,
  'week': 45,
  'month': 11,
  'year': 2018,
  'weekday': 3},
 {'start_time': 1541639702796,
  'hour': 1,
  'day': 8,
  'week': 45,
  'month': 11,
  'year': 2018,
  'weekday': 3},
 {'start_time': 1541639872796,
  'hour': 1,
  'day': 8,
  'week': 45,
  'month': 11,
  'year': 2018,
  'weekday': 3},
 {'start_time': 1541640145796,
  'hour': 1,
  'day': 8,
  'week': 45,
  'month': 11,
  'year': 2018,
  'weekday': 3},
 {'start_time': 1541640378796,
  'hour': 1,
  'day': 8,
  'week': 45,
  'month': 11,
  'year': 2018,
  'weekday': 3},
 {'start_time': 1541640498796,
  'hour': 1,
  'day': 8,
  'week': 45,
  'month': 11,
  'year': 2018,
  'weekday': 3},
 {'start_time': 1541640655796,
  'hour': 1,
  'day': 8,
  'week': 45,
  'month': 11,
  'year': 2018,
  'weekday': 3},
 {'start_time': 1541640935796,
  'hour': 1,
  'day': 8,
  'week': 45,
  'month': 11,
  'year': 2018,
  'weekday': 3},
 {'start_time': 1541641099796,
  'hour': 1,
  'day': 8,


In [23]:
insert(sql_queries.time_table_insert, time_data, cur, conn)

Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record ins

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [24]:
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Slipknot,Logged In,Aiden,M,0,Ramirez,192.57424,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Opium Of The People (Album Version),200,1541639510796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
2,Cat Stevens,Logged In,Aiden,M,1,Ramirez,170.57914,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Don't Be Shy,200,1541639702796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20


In [25]:
preparer_users = PreparerUsers() 

In [26]:
users_data = preparer_users.transform(df)

In [27]:
insert(sql_queries.user_table_insert, users_data, cur, conn)

Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record ins

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [28]:
df.head(5)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Slipknot,Logged In,Aiden,M,0,Ramirez,192.57424,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Opium Of The People (Album Version),200,1541639510796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
2,Cat Stevens,Logged In,Aiden,M,1,Ramirez,170.57914,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Don't Be Shy,200,1541639702796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
3,Collective Soul,Logged In,Aiden,M,2,Ramirez,273.47546,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Run (LP Version),200,1541639872796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
4,Taylor Swift,Logged In,Aiden,M,3,Ramirez,233.89995,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Love Story,200,1541640145796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20
5,The Dixie Cups,Logged In,Aiden,M,4,Ramirez,120.05832,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540284e+12,19,Iko Iko,200,1541640378796,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",20


In [29]:
preparer_songplays_staging = PreparerSongplaysStaging()

In [30]:
songplays_staging_data = preparer_songplays_staging.transform(df)

In [31]:
insert(sql_queries.songplay_staging_table_insert, songplays_staging_data, cur, conn)

Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record inserted successfully.
Record ins

In [63]:
query = """
SELECT songplays_staging.start_time,
       songplays_staging.user_id,
       songplays_staging.level,
       songplays_staging.song_title,
       songs_artists.song_id,
       songplays_staging.artist_name,
       songs_artists.artist_id,
       songplays_staging.session_id,
       songplays_staging.location,
       songplays_staging.user_agent
  FROM songplays_staging
  LEFT JOIN (
       SELECT songs.song_id,
              artists.artist_id,
              songs.title,
              artists.name,
              songs.duration
         FROM songs
         LEFT JOIN artists
           ON songs.artist_id = artists.artist_id)
    AS songs_artists 
    ON songplays_staging.song_title = songs_artists.title
   AND songplays_staging.artist_name = songs_artists.name
   AND songplays_staging.song_duration = songs_artists.duration;
"""

In [64]:
# query = """
# SELECT songs.song_id,
#               artists.artist_id,
#               songs.title,
#               artists.name,
#               songs.duration
#          FROM songs
#          LEFT JOIN artists
#            ON songs.artist_id = artists.artist_id;
# """

In [67]:
query = """select * from artists;"""

In [70]:
pd.read_sql(query, conn).sort_values("name")

,artist_id,name,location,lattitude,longitude
31,AR558FS1187FB45658,40 Grit,,NaN,NaN
50,AR7G5I41187FB4CE6C,Adam Ant,"London, England",NaN,NaN
40,ARI3BMM1187FB4255E,Alice Stuart,Washington,38.89910,-77.02900
44,ARL7K851187B99ACD2,Andy Andy,,NaN,NaN
21,AR3JMC51187B9AE49D,Backstreet Boys,"Orlando, FL",28.53823,-81.37739
63,AR0RCMP1187FB3F427,Billie Jo Spears,"Beaumont, TX",30.08615,-94.10158
26,AREVWGE1187B9B890A,Bitter End,Noci (BA),-13.44200,-41.99520
41,ARD842G1187B997376,Blue Rodeo,"Toronto, Ontario, Canada",43.64856,-79.38533
43,ARHHO3O1187B989413,Bob Azzam,,NaN,NaN
9,AR36F9J1187FB406F1,Bombay Rockers,Denmark,56.27609,9.51695


In [34]:
songplays_data = pd.read_sql(query, conn)

In [35]:
songplays_data

,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
0,1541677683796,72,paid,None,None,117,"Detroit-Warren-Dearborn, MI",Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...
1,1541639510796,20,paid,None,None,19,"New York-Newark-Jersey City, NY-NJ-PA","""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537...."
2,1541639702796,20,paid,None,None,19,"New York-Newark-Jersey City, NY-NJ-PA","""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537...."
3,1541639872796,20,paid,None,None,19,"New York-Newark-Jersey City, NY-NJ-PA","""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537...."
4,1541640145796,20,paid,None,None,19,"New York-Newark-Jersey City, NY-NJ-PA","""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537...."
5,1541640378796,20,paid,None,None,19,"New York-Newark-Jersey City, NY-NJ-PA","""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537...."
6,1541640498796,20,paid,None,None,19,"New York-Newark-Jersey City, NY-NJ-PA","""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537...."
7,1541640655796,20,paid,None,None,19,"New York-Newark-Jersey City, NY-NJ-PA","""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537...."
8,1541640935796,20,paid,None,None,19,"New York-Newark-Jersey City, NY-NJ-PA","""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537...."
9,1541641099796,11,free,None,None,10,"Elkhart-Goshen, IN","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."


In [ ]:
conn.commit()

In [ ]:
insert(sql_queries.songplay_table_insert, songplays_data, cur, conn)

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.